In [0]:
%run ./00_config

In [0]:
import boto3
import os
from botocore.client import Config
import pyarrow.parquet as pq
from pyspark.sql.functions import current_timestamp, monotonically_increasing_id

# Usamos las variables  en el notebook (00_config)
file_key_zones = FILE_KEY_ZONES_TABLE
s3_path_zones = f"s3a://{BUCKET_NAME}/{file_key_zones}"
nombre_tabla_zonas = BRONCE_ZONES_TABLE

try:
    print(f"Leyendo catálogo de zonas desde: {s3_path_zones}...")

    # --- LECTURA OPTIMIZADA ---
    # - header=True para tomar los nombres de columnas
    # - inferSchema=True para que Spark detecte si son números o texto
    df_zones = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("sep", ";") \
        .load(s3_path_zones)

    # --- AUDITORÍA---
    # Agregamos tanto el timestamp como el ID único que habías importado
    df_zones_final = df_zones \
        .withColumn("ingestion_timestamp", current_timestamp()) \
        .withColumn("ingestion_id", monotonically_increasing_id())

    # --- PERSISTENCIA EN DELTA ---
    # Al ser un catálogo pequeño, no necesita particionamiento, 
    df_zones_final.write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(nombre_tabla_zonas)

    print(f"✅ Tabla de zonas '{nombre_tabla_zonas}' creada con éxito.")
    display(spark.table(nombre_tabla_zonas).limit(5))

except Exception as e:
    print(f"❌ Error al cargar las zonas: {str(e)}")